# Régession linéaire corrélations et projections

On essaie de comparer une colonne des données par rapport à une autre.

In [ ]:
import pandas

epreuve = pandas.read_csv("./epreuve.txt", delim_whitespace=True)
epreuve

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
epreuve_scaled_sklearn = scaler.fit_transform(pandas.DataFrame(epreuve, dtype='float')) # sinon warning de conversion
epreuve_scaled = pandas.DataFrame(epreuve_scaled_sklearn, columns=epreuve.columns)
epreuve_scaled

# Régression

https://en.wikipedia.org/wiki/Simple_linear_regression

On va résumer nos valeurs par une droite. Mais laquelle choisir ?
On utilise la méthdoe vue en mathématiques : la régression. 

La régression tente d'expliquer une variable par rapport aux autres linéairement.

On essaye de répondre à la question suivante : Si une variable donnée était une combinaison linéaire sdes autres variables, quels seraient les coefficients ?

Il faudra aussi vérifier la validité de cette hypothèse.

Formellement, on peut alors écrire : 

Si on a $x = (x_1, \dots, x_n)$ et $y =(y_1, \dots, y_n)$ on veut trouver $a$ et $b$ tel que $y_i \sim a + b * x_i$. Donc :

$$\min_{a, b} \sum_{i=1}^n(y_i - (a + b * x_i))^2$$

Pourquoi la norme $L_2$ ? Parce que cela se dérive et que l'on trouve facilement la solution... On peut également le faire en norme $L_1$ bien sur (on le laisse en exercice).

... on dérive par rapport à $a$ et par rapport à $b$ ...

... On cherche où ces dérivées s'annulent ...

on trouve : 
    $$ a = \bar{y} - b\bar{x}$$
et :
    $$ b = \frac{cov(x, y)}{\sigma^2(x)}$$
avec : 
    $$cov(x, y) = \frac{1}{n} \sum_{i=1}^n(x_i - \bar{x})(y_i - \bar{y})$$

en python, avec le module statsmodels : https://www.statsmodels.org/stable/regression.html

Corrélations sur les données initiales :

In [ ]:
import statsmodels.api

In [ ]:
regression = statsmodels.api.OLS(epreuve_scaled['erreurs'], 
                                 statsmodels.api.add_constant(epreuve_scaled['temps'])).fit()

Les paramètres $a$ et $b$ sont alors atteignable avec les paramètres de la régression :

In [ ]:
regression.params

In [ ]:
a = regression.params.temps
b = regression.params.const

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
current_palette = sns.color_palette()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)

ax = sns.scatterplot(data=epreuve_scaled,
                     x='temps', 
                     y='erreurs',
                     ax=ax
                    )
for i in epreuve.index:
    ax.text(epreuve_scaled['temps'][i], epreuve_scaled['erreurs'][i], str(i))

ax.axvline(0, color=current_palette[0])
ax.axhline(0, color=current_palette[0])

plt.plot(ax.get_xlim(), [ x * a + b for x in ax.get_xlim()], color=current_palette[1])
    
plt.show()

Est-ce un bon résultat ? On va calculer l'erreur. C'est à dire calculer $ \sum_{i=1}^n(y_i - (a * x_i + b))^2$ avec nos données et les $a$ et $b$ minimaux. On trouve que cela vaut :
$$ E = \sigma^2(y)(1 - (\frac{cov(x, y)}{\sigma(x)\sigma(y)})^2)$$

La qualité de la régression sera dnc d'autant meilleur que E sera petit. On doit donc avoir:
- un petit $\sigma^2(y)$ (que l'on peut supposer $>0$)
- un gros $(\frac{cov(x, y)}{\sigma(x)\sigma(y)})^2$

Au passage, comme $E\geq 0$ on a forcément que $0\leq (\frac{cov(x, y)}{\sigma(x)\sigma(y)})^2 \leq 1$ et que les points sont alignées ($E = 0$) si et seulement si $(\frac{cov(x, y)}{\sigma(x)\sigma(y)})^2 = 1$. C'est la musure principale pour la qualité de la régression on l'appelle ainsi coefficient de régression linéaire et on note :
$$ r(x, y)^2 = (\frac{cov(x, y)}{\sigma(x)\sigma(y)})^2$$

Avec pandas, on trouve facilement le coefficient de corrélation : 

In [ ]:
epreuve_scaled.corr()

Et le $r^2$ : 

In [ ]:
#r^2 

0.766547 ** 2

On retrouve bien que chaque axe est parfaitement linéairement corrélé avec lui-même (ouf) et que la corrélation linéaire d'une axe avec l'autre est identique

On retrouve ce coefficient, et plein d'autre dans :

In [ ]:
regression.summary()

Le coefficient de corrélation est identique sur les données centrées/réduite et sur les données initiales : les données ne sont pas plus corrélées linéairement entres elle. C'est normale puisque centrer et réduire est une transformation linéaire de nos données.

En revanche, la pente (qui dépend de la variance) et la constante (qui dépend de la moyenne) changent, en particulier la droite ne passe maintenant plus par l'origine (la moyenne n'est pas nulle).

In [ ]:
(statsmodels.api.OLS(epreuve['erreurs'], 
                    statsmodels.api.add_constant(epreuve['temps']))
    .fit()
    .summary())

# corrélation

Dans l'optique d'une analyse des données, il est important de considérer le **coéfficient de corrélation** $r(x, y)$ entre deux colonnes $x$ et $y$.

Il a plein de propriétés :

* $r(x, y) = r(y, x)$
* $-1 \leq r(x, y) \leq 1$
* si $\vert r(x, y) \vert = 1$ alors $x$ et $y$ sont linéairement dépendant

On s'en sert comme ça :

* si $r(x, y)$ est proche de 1, alors :
    * $x$ et $y$ sont corrélés positivement (presque linéairement dépendant) 
    * lorsque $x$ augmente alors $y$ augmente et lorsque $x$ diminue alors $y$ diminue
    * lorsque $y$ augmente alors $x$ augmente et lorsque $y$ diminue alors $x$ diminue
* si $r(x, y)$ est proche de -1, alors 
    * $x$ et $y$ sont corrélés négativement (presque linéairement dépendant) 
    * lorsque $x$ augmente alors $y$ diminue et lorsque $x$ diminue alors $y$ augmente
    * lorsque $y$ augmente alors $x$ diminue et lorsque $y$ diminue alors $x$ augmente
    

# projection

Il n'y a aucune raison de privilégier une variable plutôt qu'une autre ici. Que donne la regression de l'erreur par le temps ?

In [ ]:
regression2 = statsmodels.api.OLS(epreuve_scaled['temps'], 
                                 statsmodels.api.add_constant(epreuve_scaled['erreurs'])).fit()

regression2.summary()

In [ ]:
a_prim = regression2.params.erreurs
b_prim = regression2.params.const

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)

ax = sns.scatterplot(data=epreuve_scaled,
                     x='temps', 
                     y='erreurs',
                     ax=ax
                    )
for i in epreuve.index:
    ax.text(epreuve_scaled['temps'][i], epreuve_scaled['erreurs'][i], str(i))

ax.axvline(0)
ax.axhline(0)

plt.plot([ x * a_prim + b_prim for x in ax.get_ylim()], ax.get_ylim(), color=current_palette[2])
plt.plot(ax.get_xlim(), [ x * a + b for x in ax.get_xlim()], color=current_palette[1])
    
plt.show()

Comme $y = ax + b$ et $x = a'y + b'$, les deux droites ne peuvent être égales que si $a = \frac{1}{a'} $, c'est à dire que $r(x, y)^2 = 1$ : les deux droite ne sont égales que si et seulement si es points sont déjà alignés.

Il nous faut trouver un autre moyen d'approximer nos points par une droite car on ne peut ici privilégier l'un ou l'autre des axes. Idée, minimiser la projection. En 2D, cela revient à prendre à bissectrice.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)

ax = sns.scatterplot(data=epreuve_scaled,
                     x='temps', 
                     y='erreurs',
                     ax=ax
                    )
for i in epreuve.index:
    ax.text(epreuve_scaled['temps'][i], epreuve_scaled['erreurs'][i], str(i))

ax.axvline(0)
ax.axhline(0)

plt.plot([ x * a_prim + b_prim for x in ax.get_ylim()], ax.get_ylim(), color=current_palette[2])
plt.plot(ax.get_xlim(), [.5 * (a + 1/a_prim) * x + b for x in ax.get_xlim()], color=current_palette[7], linestyle="--")
plt.plot(ax.get_xlim(), [ x * a + b for x in ax.get_xlim()], color=current_palette[1])
    
plt.show()

On en reparlera lorsque l'on étudiera l'analyse en composantes principales.

La droite de projection s'interprète comme une **tendance** dans les données

# seaborn

Seaborn nous aide et permet de représenter facilement les régressions avec [regplot](https://seaborn.pydata.org/generated/seaborn.regplot.html).

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)

sns.regplot(data=epreuve_scaled,
            x='temps',
            y='erreurs',
            ax=ax
          )
ax.axvline(0)
ax.axhline(0)

plt.show()

La zone bleue est l'erreur.  Toutes les deroites dans cette zone peuvent être considérées comme des droites de régression (à 95%)

**A lire** :  https://serialmentor.com/dataviz/visualizing-uncertainty.html#uncertainty-curve-fits tout le livre qui est supper et cette partie en particulier. 

## Matrice de corrélation

Lorsque l'on a plus de 2 variables, la matrice de corrélation est difficile à lire d'un seul coup d'oeil. 

In [ ]:
import seaborn as sns

iris = sns.load_dataset('iris')
iris

In [ ]:
iris.corr()

On peut alors utiliser une [heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) pour rendre tout ça plus visuel.

On a utilisé la palette de couleur `'coolwarm'` de matplotlib. La gestion des [palettes de couleur en matplotlib](https://matplotlib.org/stable/tutorials/colors/colormaps.html) est — comme tout ce qui a trait à matplotlib — inutilement compliquée mais farfois on ne peut pas faire autrement que de l'utiliser (et de pester).

In [ ]:
data = iris.corr()

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(data, 
            annot=True, 
            cmap='coolwarm',
            vmin=-1, vmax=1)
plt.show()

Ou les corrélations au carré (attention à `*` qui n'est [pas la multiplication de matrice](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.multiply.html) :

In [ ]:
data = iris.corr() * iris.corr()

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(data, 
            annot=True, 
            cmap='coolwarm',
            vmin=0, vmax=1)
plt.show()

On voit bien les colonnes corrélées entre elles (proches de 1, rouge) et celles qui ne le sont pas (proche de 0, en bleu)

Si on veut voir les valeurs absolues de la corrélation, on peut utiliser une [map](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html). Cela consiste à appliquer une fonction donnée à chaque élément du dataframe (ici la fonction [abs](https://docs.python.org/3/library/functions.html#abs)).

In [ ]:
# sans la map
iris.corr()

In [ ]:
# avec la map 
iris.corr().applymap(abs)

In [ ]:
data = iris.corr().applymap(abs)

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(data, 
            annot=True, 
            cmap='coolwarm',
            vmin=0, vmax=1)
plt.show()